In [1]:
import pickle
import pandas as pd
import numpy as np
import math
import random
from tqdm import tqdm
from collections import defaultdict

## Load Data

In [3]:
with open('song_user.dict', 'rb') as handle:
    unserialized_song_user = pickle.load(handle) #dict
    
with open('user_song.dict', 'rb') as handle:
    unserialized_user_song = pickle.load(handle) #dict
    
with open('song_person.dict', 'rb') as handle:
    unserialized_song_person = pickle.load(handle) #dict
    
with open('person_song.dict', 'rb') as handle:
    unserialized_person_song = pickle.load(handle) #dict

with open('song_id.txt', 'rb') as handle:
    unserialized_song_id = pickle.load(handle) #numpy.ndarray
    
with open('user_id.txt', 'rb') as handle:
    unserialized_user_id = pickle.load(handle) #numpy.ndarray
    
with open('person_id.txt', 'rb') as handle:
    unserialized_person_id = pickle.load(handle) #numpy.ndarray

In [4]:
# turn the relation dictionaries to defaultdict that return [] for a node that is not in the keys
unserialized_song_user = defaultdict(list, unserialized_song_user)
unserialized_song_person = defaultdict(list, unserialized_song_person)
unserialized_user_song = defaultdict(list, unserialized_user_song)
unserialized_person_song = defaultdict(list, unserialized_person_song)

## Sort Nodes By Degree

In [5]:
# key: song, value: number of users listening to it + number of person relating to its creation
song_degree_dict = {}
for (k,v) in unserialized_song_user.items():
    song_degree_dict[k] = v
for (k,v) in unserialized_song_person.items():
    if k in song_degree_dict.keys():
        song_degree_dict[k] = song_degree_dict[k] + v
    else:
        song_degree_dict[k] = v
song_degree = [(k,len(v)) for (k,v) in song_degree_dict.items()]
print('There are %d songs in the network' % len(song_degree))

# sort by degree in descending order
song_degree.sort(key = lambda x : -x[1])

There are 2296372 songs in the network


In [6]:
# key: person, value: number of songs they create
person_degree = [(k,len(v)) for (k,v) in unserialized_person_song.items()]
print('There are %d persons in the network' % len(person_degree))

# sort by degree in descending order
person_degree.sort(key = lambda x : -x[1])

There are 527630 persons in the network


In [7]:
# key: user, value: number of songs they listen to
user_degree = [(k,len(v)) for (k,v) in unserialized_user_song.items()]
print('There are %d users in the network' % len(user_degree))

# sort by degree in descending order
user_degree.sort(key = lambda x : -x[1])

There are 30755 users in the network


## Construct Subnetworks

In [9]:
def find_edges(nodes_list):
    # (node1, node2) and (node2, node1) both exist
    edges_type1 = [] # a list of pairs (song, user)
    edges_type2 = [] # a list of pairs (song, person)
    edges_type3 = [] # a list of pairs (user, song)
    edges_type4 = [] # a list of pairs (person, song)
    nodes_set = set(nodes_list)
    
    for i in tqdm(nodes_set): # (node1, node2) and (node2, node1) both exist
        connect_1 = set(unserialized_song_user[i]).intersection(nodes_set)
        for j in connect_1:
            edges_type1.append((i,j))

        connect_2 = set(unserialized_song_person[i]).intersection(nodes_set)
        for j in connect_2:
            edges_type2.append((i,j))

        connect_3 = set(unserialized_user_song[i]).intersection(nodes_set)
        for j in connect_3:
            edges_type3.append((i,j))

        connect_4 = set(unserialized_person_song[i]).intersection(nodes_set)
        for j in connect_4:
            edges_type4.append((i,j))

    return edges_type1, edges_type2, edges_type3, edges_type4

### Construct Sparse Subnet (not using random sampling) 

In [10]:
# construct sparse subnetwork
sparse_nodes_percent = 0.1

# find the nodes
print('finding the nodes...')
sparse_song_nodes_holder = song_degree[-int(len(song_degree)*sparse_nodes_percent):] #song_id is the first item in the tuple element of the returned list
sparse_song_nodes = [node_holder[0] for node_holder in sparse_song_nodes_holder]

sparse_user_nodes_holder = user_degree[-int(len(user_degree)*sparse_nodes_percent):]
sparse_user_nodes = [node_holder[0] for node_holder in sparse_user_nodes_holder]

sparse_person_nodes_holder = person_degree[-int(len(person_degree)*sparse_nodes_percent):]
sparse_person_nodes = [node_holder[0] for node_holder in sparse_person_nodes_holder]

sparse_nodes = sparse_song_nodes + sparse_user_nodes + sparse_person_nodes

print('sparse nodes: %d songs, %d users, %d persons.' % (len(sparse_song_nodes),\
                                                                                         len(sparse_user_nodes), \
                                                                                         len(sparse_person_nodes)))
print('total number sparse nodes: %d.' % len(sparse_nodes))

finding the nodes...
sparse nodes: 229637 songs, 3075 users, 52763 persons.
total number sparse nodes: 285475.


In [11]:
# construct sparse subnetwork
# build the subnetwork --> find the edges
# (node1, node2) and (node2, node1) both exist
sparse_edges_type1, sparse_edges_type2, sparse_edges_type3, sparse_edges_type4 = find_edges(sparse_nodes)
sparse_edges = sparse_edges_type1 + sparse_edges_type2 + sparse_edges_type3 + sparse_edges_type4
print('num edges: ', len(sparse_edges))

100%|██████████| 285473/285473 [00:04<00:00, 60058.83it/s] 

num edges:  24068


### Construct Sparse Subnet (random sampling) 

In [12]:
# construct sparse subnetwork
rs_sparse_nodes_percent = 0.1

# find the nodes
print('finding the nodes...')
rs_sparse_song_nodes_holder = random.sample(song_degree, int(len(song_degree)*rs_sparse_nodes_percent)) #song_id is the first item in the tuple element of the returned list
rs_sparse_song_nodes = [node_holder[0] for node_holder in rs_sparse_song_nodes_holder]

rs_sparse_user_nodes_holder = random.sample(user_degree, int(len(user_degree)*rs_sparse_nodes_percent))
rs_sparse_user_nodes = [node_holder[0] for node_holder in rs_sparse_user_nodes_holder]

rs_sparse_person_nodes_holder = random.sample(person_degree, int(len(person_degree)*rs_sparse_nodes_percent))
rs_sparse_person_nodes = [node_holder[0] for node_holder in rs_sparse_person_nodes_holder]

rs_sparse_nodes = rs_sparse_song_nodes + rs_sparse_user_nodes + rs_sparse_person_nodes

print('rs sparse nodes: %d songs, %d users, %d persons.' % (len(rs_sparse_song_nodes),\
                                                                                         len(rs_sparse_user_nodes), \
                                                                                         len(rs_sparse_person_nodes)))
print('total number rs sparse nodes: %d.' % len(rs_sparse_nodes))

finding the nodes...
rs sparse nodes: 229637 songs, 3075 users, 52763 persons.
total number rs sparse nodes: 285475.


In [13]:
# construct sparse subnetwork
# build the subnetwork --> find the edges
# (node1, node2) and (node2, node1) both exist
rs_sparse_edges_type1, rs_sparse_edges_type2, rs_sparse_edges_type3, rs_sparse_edges_type4 = \
                                                                                                                    find_edges(rs_sparse_nodes)

rs_sparse_edges = rs_sparse_edges_type1 + rs_sparse_edges_type2 + rs_sparse_edges_type3 + rs_sparse_edges_type4
print('num edges: ', len(rs_sparse_edges))

100%|██████████| 285473/285473 [00:03<00:00, 72203.86it/s] 

num edges:  258512


### Construct Dense Subnet (not using random sampling) 

In [14]:
# construct dense subnetwork
dense_nodes_percent = 0.1

# find the nodes
print('finding the nodes...')
dense_song_nodes_holder = song_degree[:int(len(song_degree)*dense_nodes_percent)] #song_id is the first item in the tuple element of the returned list
dense_song_nodes = [node_holder[0] for node_holder in dense_song_nodes_holder]

dense_user_nodes_holder = user_degree[:int(len(user_degree)*dense_nodes_percent)]
dense_user_nodes = [node_holder[0] for node_holder in dense_user_nodes_holder]

dense_person_nodes_holder = person_degree[:int(len(person_degree)*dense_nodes_percent)]
dense_person_nodes = [node_holder[0] for node_holder in dense_person_nodes_holder]

dense_nodes = dense_song_nodes + dense_user_nodes + dense_person_nodes

print('dense nodes: %d songs, %d users, %d persons.' % (len(dense_song_nodes),\
                                                                                         len(dense_user_nodes), \
                                                                                         len(dense_person_nodes)))
print('total number dense nodes: %d.' % len(dense_nodes))

finding the nodes...
dense nodes: 229637 songs, 3075 users, 52763 persons.
total number dense nodes: 285475.


In [16]:
# construct sparse subnetwork
# build the subnetwork --> find the edges
# (node1, node2) and (node2, node1) both exist
dense_edges_type1, dense_edges_type2, dense_edges_type3, dense_edges_type4 = find_edges(dense_nodes)
dense_edges = dense_edges_type1 + dense_edges_type2 + dense_edges_type3 + dense_edges_type4
print('num edges: ', len(dense_edges))

100%|██████████| 285470/285470 [00:10<00:00, 26989.40it/s]


num edges:  6967762


## Construct Entire Network

In [ ]:
# comment out because takes too long
'''
# construct entire network
# find the nodes
print('finding the nodes...')
entire_net_song_nodes = [node_holder[0] for node_holder in song_degree]
entire_net_user_nodes = [node_holder[0] for node_holder in user_degree]
entire_net_person_nodes = [node_holder[0] for node_holder in person_degree]
entire_net_nodes = entire_net_song_nodes + entire_net_user_nodes + entire_net_person_nodes

# build the subnetwork --> find the edges
print('finding the edges...')
entire_net_edge = [] # a list of pairs (node1, node2)
connect = [] # a list of nodes that the node1 should connect to
for i in entire_net_nodes: # (node1, node2) and (node2, node1) both exist
    if i in unserialized_song_user:
        connect = unserialized_song_user[i]
        if i in unserialized_song_person:
            connect.extend(unserialized_song_person[i])
    elif i in unserialized_song_person:
        connect = unserialized_song_person[i]
        if i in unserialized_song_user:
            connect.extend(unserialized_song_user[i])
    elif i in unserialized_user_song:
        connect = unserialized_user_song[i]
    elif i in unserialized_person_song:
        connect = unserialized_person_song[i]
    else:
        print('Error: key error!')
    new_edges = [(i, j) for j in connect]
    entire_net_edge.extend(new_edges)
'''

## Compute Statistics & Density Heuristic

In [18]:
# compute the density heuristic 
sparse_net_density = len(sparse_edges) / (len(sparse_song_nodes) * \
                                                                (len(sparse_user_nodes) + len(sparse_person_nodes)))
print('sparse subnet density heuristic: ', sparse_net_density)

# number of nodes
print('sparse subnet has %d nodes' % len(sparse_nodes))

# number of edges 
print('sparse subnet has %d edges' % len(sparse_edges))

sparse subnet density heuristic:  1.8770173365290796e-06
sparse subnet has 285475 nodes
sparse subnet has 24068 edges


### Results:
sparse subnet density heuristic:  1.8770173365290796e-06

sparse subnet has 285475 nodes

sparse subnet has 24068 edges

In [19]:
# compute the density heuristic 
rs_sparse_net_density = len(rs_sparse_edges) / (len(rs_sparse_song_nodes) * \
                                                                        (len(rs_sparse_user_nodes) + len(rs_sparse_person_nodes)))
print('(randomly sampled) sparse subnet density heuristic: ', rs_sparse_net_density)

# number of nodes
print('(randomly sampled) sparse subnet has %d nodes' % len(rs_sparse_nodes))

# number of edges 
print('(randomly sampled) sparse subnet has %d edges' % len(rs_sparse_edges))

(randomly sampled) sparse subnet density heuristic:  2.016085697610127e-05
(randomly sampled) sparse subnet has 285475 nodes
(randomly sampled) sparse subnet has 258512 edges


### Results:
rs sparse subnet density heuristic:  1.8549233107920558e-05

rs sparse subnet has 285475 nodes

rs sparse subnet has 237847 edges

In [20]:
# compute the density heuristic 
dense_net_density = len(dense_edges) / (len(dense_song_nodes) * \
                                                                (len(dense_user_nodes) + len(dense_person_nodes)))
print('dense subnet density heuristic: ', dense_net_density)

# number of nodes
print('dense subnet has %d nodes' % len(dense_nodes))

# number of edges 
print('dense subnet has %d edges' % len(dense_edges))

dense subnet density heuristic:  0.0005434024460199657
dense subnet has 285475 nodes
dense subnet has 6967762 edges


### Results
dense subnet density heuristic:  0.0005436736885950217

dense subnet has 285475 nodes

dense subnet has 6971240 edges

In [ ]:
'''
# compute the density heuristic 
entire_net_density = len(entire_net_edge) / (len(entire_net_song_nodes) * \
                                                                (len(entire_net_user_nodes) + len(entire_net_person_nodes)))
print('entire subnet density heuristic: ', entire_net_density)

# number of nodes
print('entire subnet has %d nodes' % len(entire_net_nodes))

# number of edges 
print('entire subnet has %d nodes' % len(entire_net_edge))
'''

## Export Subnetworks

In [81]:
def export_subnetwork(network_type, all_nodes, person_nodes, song_nodes, user_nodes, \
                                 all_edges, edges_type1, edges_type2, edges_type3, edges_type4):
    #network_type is a string, the other params are lists and dictionaries 
    
    # <NETWORK TYPE>_nodes.txt
    all_nodes_np = np.array(all_nodes) # numpy array of nodes of sparse network
    with open('%s_nodes.txt' % network_type, 'wb') as handle:
        pickle.dump(all_nodes_np, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # <NETWORK TYPE>_edges.txt
    # key: source, value: target
    # note that node1 -> node2 and node2 -> node1 are both present in this dictionary
    all_edges_dict = {pair[0]:pair[1] for pair in all_edges}
    with open('%s_edges.dict' % network_type, 'wb') as handle:
        pickle.dump(all_edges_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    # <NETWORK TYPE>_song_nodes.txt
    sparse_song_nodes_np = np.array(sparse_song_nodes) # numpy array of nodes of sparse network
    with open('sparse_song_nodes.txt', 'wb') as handle:
        pickle.dump(sparse_song_nodes_np, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # <NETWORK TYPE>_user_nodes.txt
    sparse_user_nodes_np = np.array(sparse_user_nodes) # numpy array of nodes of sparse network
    with open('sparse_user_nodes.txt', 'wb') as handle:
        pickle.dump(sparse_user_nodes_np, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # <NETWORK TYPE>_person_nodes.txt
    sparse_person_nodes_np = np.array(sparse_person_nodes) # numpy array of nodes of sparse network
    with open('sparse_person_nodes.txt', 'wb') as handle:
        pickle.dump(sparse_person_nodes_np, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    # <NETWORK TYPE>_song_user_edges.dict
    # key: song, value: a list of users
    song_user_dict = defaultdict(list)
    for edge in edges_type1:
        song = edge[0]
        user = edge[1]
        song_user_dict[song].append(user)
    song_user_dict = dict(song_user_dict)
    with open('%s_song_user_edges.dict' % network_type, 'wb') as handle:
        pickle.dump(song_user_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    # <NETWORK TYPE>_song_person_edges.dict
    # key: song, value: a list of persons
    song_person_dict = defaultdict(list)
    for edge in edges_type2:
        song = edge[0]
        person = edge[1]
        song_person_dict[song].append(person)
    song_person_dict = dict(song_person_dict)
    with open('%s_song_person_edges.dict' % network_type, 'wb') as handle:
        pickle.dump(song_person_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    # <NETWORK TYPE>_user_song_edges.dict
    # key: user, value: a list of songs
    user_song_dict = defaultdict(list)
    for edge in edges_type3:
        user = edge[0]
        song = edge[1]
        user_song_dict[song].append(song)
    user_song_dict = dict(user_song_dict)
    with open('%s_user_song_edges.dict' % network_type, 'wb') as handle:
        pickle.dump(user_song_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    # <NETWORK TYPE>_person_song_edges.dict
    # key: person, value: a list of songs
    person_song_dict = defaultdict(list)
    for edge in edges_type4:
        person = edge[0]
        song = edge[1]
        person_song_dict[song].append(song)
    person_song_dict = dict(person_song_dict)
    with open('%s_person_song_edges.dict' % network_type, 'wb') as handle:
        pickle.dump(person_song_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [82]:
export_subnetwork('sparse', sparse_nodes, sparse_person_nodes, \
                            sparse_song_nodes, sparse_user_nodes, \
                            sparse_edges, sparse_edges_type1, sparse_edges_type2, \
                            sparse_edges_type3, sparse_edges_type4)

In [83]:
export_subnetwork('rs_sparse', rs_sparse_nodes, rs_sparse_person_nodes, \
                            rs_sparse_song_nodes, rs_sparse_user_nodes, \
                            rs_sparse_edges, rs_sparse_edges_type1, rs_sparse_edges_type2, \
                            rs_sparse_edges_type3, rs_sparse_edges_type4)

In [ ]:
export_subnetwork('dense', dense_nodes, dense_person_nodes, \
                            dense_song_nodes, dense_user_nodes, \
                            dense_edges, dense_edges_type1, dense_edges_type2, \
                            dense_edges_type3, dense_edges_type4)

## Test Loading The Saved Subnets

In [56]:
with open('sparse_nodes.txt', 'rb') as handle:
    unpickled_sparse_nodes = pickle.load(handle)
unpickled_sparse_net_nodes[:5]
len(unpickled_sparse_nodes)

285475

In [57]:
with open('sparse_edges.dict', 'rb') as handle:
    unpickled_sparse_edges = pickle.load(handle)
len(unpickled_sparse_edges)

24068

In [60]:
with open('sparse_person_nodes.txt', 'rb') as handle:
    unpickled_sparse_person_nodes = pickle.load(handle)
unpickled_sparse_person_nodes[:5]
len(unpickled_sparse_net_nodes)

285475

In [80]:
with open('rs_sparse_song_user_edges.dict', 'rb') as handle:
    unpickled_rs_sparse_song_user_edges = pickle.load(handle)
len(unpickled_rs_sparse_song_user_edges)

11948

In [84]:
with open('rs_sparse_song_person_edges.dict', 'rb') as handle:
    unpickled_rs_sparse_song_user_edges = pickle.load(handle)
len(unpickled_rs_sparse_song_user_edges)

38330

In [85]:
with open('rs_sparse_user_song_edges.dict', 'rb') as handle:
    unpickled_rs_sparse_song_user_edges = pickle.load(handle)
len(unpickled_rs_sparse_song_user_edges)

11948

In [86]:
with open('rs_sparse_person_song_edges.dict', 'rb') as handle:
    unpickled_rs_sparse_song_user_edges = pickle.load(handle)
len(unpickled_rs_sparse_song_user_edges)

38330